In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [86]:
def ReadObs():
    # 读取sdwa1000.18o文件，获得n*2的矩阵。
    File_pointer = open('sdwa1000.18o', 'r')
    OBS = []
    
    while True:
        line = File_pointer.readline()
        
        if line[0] == '>':
            # 判断时间，文件中时间部分的第一个字符为>
            year = int(line[2:6])
            month = int(line[7:9])
            day = int(line[10:12])
            hour = int(line[13:15])
            minute = int(line[16:18])
            second = float(line[19:28])
            
            # 以上分别为年月日时分秒，接下来挑选自己所需的时间点（2018年4月10日0时35分0秒）
            if (year == 2018) and (month == 4) and (day == 10) and (hour == 0) and (minute == 35) and (second == 0):
                number = int(line[33:35])
                # 保存卫星的数量，下面写for循环，挑选GPS的数据
                
                for i in range(number):
                    line = File_pointer.readline()
                    # 在这里更新一行的数据
                    
                    if line[0] == 'G':
                        # 挑选GPS数据
                        OBS.append([float(line[5:18]), float(line[19:34])])
                
                break
        elif not line:
            # 达到文件末尾，退出循环
            break
    
    File_pointer.close()
    # 关闭打开的文件
    return OBS

In [87]:
ReadObs()

[[21807328.555, 21807336.215],
 [20649902.414, 20649911.441],
 [22049662.719, 22049669.738],
 [20748971.289, 20748981.398],
 [20535729.711, 20535739.953],
 [24093282.586, 24093293.199],
 [21553535.555, 21553542.781]]

In [95]:
def ReadSP3():
    # 读取igs19962.sp3文件，获得到4个n*10矩阵
    File_pointer1 = open('igs19962.sp3', 'r') # 32个卫星
    SData1 = [[0] * 10 for _ in range(33)]
    SData2 = [[0] * 10 for _ in range(33)]
    SData3 = [[0] * 10 for _ in range(33)]
    SData4 = [[0] * 10 for _ in range(33)]
    
    while True:
        line = File_pointer1.readline()
        
        if line[0] == '*':
            # 筛选开头为*的行数据，为一组数据的开始
            for i in range(10):  # 从头开始，根据题目总共要选前十
                for j in range(33):
                    line = File_pointer1.readline()  # 更新数据
                    if line[0] == 'P':
                        SData1[j][i] = float(line[4:18])
                        SData2[j][i] = float(line[18:32])
                        SData3[j][i] = float(line[32:46])
                        SData4[j][i] = float(line[46:60])
                        # 1.2.3.4分别为x.y.z.t的32*10数组。
            
            break
        elif not line:
            # 达到文件末尾，退出循环
            break
    
    File_pointer1.close()
    return SData1, SData2, SData3, SData4
# 当心有无缺失的pg卫星

In [96]:
ReadSP3()

([[15598.519741,
   16423.836531,
   17325.471332,
   18270.933329,
   19223.069652,
   20141.532092,
   20984.365736,
   21709.656736,
   22277.175761,
   22649.955411],
  [-14330.526557,
   -14385.735718,
   -14337.783422,
   -14152.646746,
   -13800.074397,
   -13254.839057,
   -12497.783755,
   -11516.631759,
   -10306.539026,
   -8870.377598],
  [20584.70139,
   19625.961318,
   18622.824856,
   17616.246107,
   16644.206642,
   15740.248219,
   14932.193212,
   14241.102127,
   13680.508049,
   13255.956852],
  [-3482.562617,
   -4731.939838,
   -5766.153082,
   -6591.444171,
   -7222.156035,
   -7680.032886,
   -7993.220125,
   -8194.99887,
   -8322.302319,
   -8414.072129],
  [-11547.606797,
   -10031.854564,
   -8641.777513,
   -7396.3846,
   -6307.087631,
   -5377.388508,
   -4602.877187,
   -3971.543377,
   -3464.392912,
   -3056.347459],
  [-5956.130786,
   -5510.633847,
   -4908.292258,
   -4125.634472,
   -3146.132027,
   -1961.032142,
   -569.888245,
   1019.231931,
   2

In [31]:
def lagrange(xx, yy, x):
    n = len(xx)
    A = np.ones(n)  # 初始化各节点基函数
    for j in range(n):
        for i in range(n):
            if i != j:
                A[j] = A[j] * (x - xx[i]) / (xx[j] - xx[i])  # 连乘
    phi = np.sum(A * yy)  # 连加
    return phi

In [100]:
def Interpolation(rho):
    times = np.array([0, 15, 30, 45, 60, 75, 90, 105, 120, 135])
    t = 35 # 在35min时进行差值
    # 这意味着希望以时间点35作为目标点，通过拉格朗日插值方法来估计在这个时间点上的卫星数据
    SData1, SData2, SData3, SData4 = ReadSP3()
    # 选出与obs文件相对应的卫星（35min时yyO文件的GPS卫星编号-1）
    sdata1 = np.array(SData1)[[30, 31, 13, 9, 24, 23, 11], :] # 选择特定的行和所有的列
    sdata2 = np.array(SData2)[[30, 31, 13, 9, 24, 23, 11], :]
    sdata3 = np.array(SData3)[[30, 31, 13, 9, 24, 23, 11], :]
    sdata4 = np.array(SData4)[[30, 31, 13, 9, 24, 23, 11], :]
    SData = np.zeros((7, 4)) # 初始化
    for i in range(7):
        SData[i, 0] = lagrange(times, sdata1[i, :], t - (rho / 299792458) / 60)
        SData[i, 1] = lagrange(times, sdata2[i, :], t - (rho / 299792458) / 60)
        SData[i, 2] = lagrange(times, sdata3[i, :], t - (rho / 299792458) / 60)
        SData[i, 3] = lagrange(times, sdata4[i, :], t - (rho / 299792458) / 60)
# t - (rho / 299792458) / 60：获得一个修正后的时间 -> 钟差
    SData[:, 0:3] = SData[:, 0:3] * 1000  # 换算单位km到m
    SData[:, 3] = SData[:, 3] * 1.0e-06  # 换算单位us到s

    return SData

In [140]:
def Positioning():
    OBS = ReadObs()
    SatData = Interpolation(0) # 第一次进行插值
    x1 = SatData[:, 0]
    y1 = SatData[:, 1]
    z1 = SatData[:, 2]
    t1 = SatData[:, 3]
    p = np.array([0, 0, 0, 0])
    dX = np.array([1, 1, 1, 1])
    L = [0] * 7
    A = np.zeros((7, 4))
    c = 299792458
    f1 = 1575.42 # C1C
    f2 = 1227.6 # C2W
    while np.sqrt(np.sum(dX**2)) >= 0.0001:  # 判断dX的改变小于0.0001
        for s in range(7):
            P1 = OBS[s][0]
            P2 = OBS[s][1]
            PC = (f1**2 * P1) / (f1**2 - f2**2) - (f2**2 * P2) / (f1**2 - f2**2)
            rho = np.sqrt((x1[s] - p[0])**2 + (y1[s] - p[1])**2 + (z1[s] - p[2])**2)  # rho是公式中的ρ
            SatData = Interpolation(rho)  # 对时间进行修正，得到新的ρ
            x1 = SatData[:, 0]
            y1 = SatData[:, 1]
            z1 = SatData[:, 2]
            t1 = SatData[:, 3]
            rho = np.sqrt((x1[s] - p[0])**2 + (y1[s] - p[1])**2 + (z1[s] - p[2])**2)
            ax = (p[0] - x1[s]) / rho
            ay = (p[1] - y1[s]) / rho
            az = (p[2] - z1[s]) / rho
            adt = c
            L[s] = PC - rho - c * p[3] + c * t1[s]
            A[s, 0]=ax;
            A[s, 1]=ay;
            A[s, 2]=az;
            A[s, 3]=adt; 

        Q = np.linalg.inv(A.T @ A)
        dX = Q @ (A.T @ L)
        for i in range(4):
            p[i] += dX[i]

    return p

In [141]:
Positioning()

KeyboardInterrupt: 